TITANIC DATASET - ML - NAIVE BAYES
==================================

In [10]:
%load_ext autoreload
%autoreload 2

import mlflow
import pickle
import pandas as pd 
import numpy as np 
from datetime import datetime
from functions import get_metrics

from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, KFold

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configs

In [11]:
EXPERIMENT_TITLE = "Titanic Dataset Analyzes"

run_description = """
### Descrição

Implementação usando [Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
"""

tags = {
    "date": datetime.now(),
    "author": "Pablo Veinberg",
    "version": 1.0,
    "envoronment": "local",
    "mlflow.note.content": run_description,
    "mlflow.runName": "Naive Bayes",
    "data_source": "./../datasets/silver/train-encoded-not-normalize.parquet",
    "train_test_dataset": "./../datasets/silver/titanic-train-test-data.pkl"
}


In [12]:
with open(tags['train_test_dataset'], 'rb') as file:
    X_train, X_test, y_train, y_test = pickle.load(file)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 8), (179, 8), (712,), (179,))

## Process

In [13]:
mlflow.set_experiment(EXPERIMENT_TITLE)

dataset = pd.read_parquet(tags['data_source'])
mlflow_dataset = mlflow.data.from_pandas(dataset, \
                                            source=tags['data_source'], \
                                            name="Titanic Dataset")

Traceback (most recent call last):
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 302, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 395, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1303, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1296, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 303, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.e

In [14]:
params = {
    "token": np.random.randint(10_000,high=99_000),
    "model_params": {'priors': [dataset['Survived'].value_counts(normalize=True).values, [.5, .5]]}
}

In [15]:

with mlflow.start_run():

    # Settings
    mlflow.log_input(mlflow_dataset)
    mlflow.set_tags(tags)
    mlflow.log_params(params)

    grid_search = GridSearchCV(estimator=GaussianNB(), \
                               param_grid=params["model_params"])
    
    grid_search.fit(X_train, y_train)
    
    model = GaussianNB(**grid_search.best_params_)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    plt_matrix_confusion_image_path = f"./../results/naive_bayes_m_confusion_{params['token']}.png"
    
    metrics = get_metrics(y_test, y_pred, \
                        plt_matrix_confusion_image_path, \
                        "Titanic - Naive Bayes Confusion Matrix")
    
    mlflow.log_param("best_params", grid_search.best_params_)
    mlflow.log_metrics(metrics)
    mlflow.log_artifact(plt_matrix_confusion_image_path)

mlflow.end_run()


/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/data/pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)
